# **Azure DevOps CI/CD Pipeline for Azure Data Factory (ADF)**
This document outlines the steps to set up **CI/CD pipelines in Azure DevOps** for deploying **Azure Data Factory (ADF)** across **Development (Dev), Quality Assurance (QA), and Production (Prod) environments**.

## **1. Prerequisites**
Before proceeding, ensure you have:
- **Azure Subscription** with three ADF instances:
  - `adf-dev`
  - `adf-qa`
  - `adf-prod`
- **Azure DevOps repository** configured with ADF source code.
- **Service Connection** in Azure DevOps for deployment.
- **Installed Azure CLI** (optional for automation).

---

## **2. Connecting ADF to Azure DevOps**
### **Step 1: Link ADF to Azure DevOps Repo**
1. Open **Azure Data Factory** (Dev Instance).
2. Navigate to **Manage** (⚙️) > **Git Configuration**.
3. Click **Configure Git** and enter:
   - **Azure DevOps Git**
   - **Project and Repository**
   - **Collaboration branch**: `main`
   - **Publish branch**: `adf_publish`
4. Click **Apply**.

Repeat for **QA and Prod instances**, ensuring each is linked to a separate environment branch.

---

## **3. Setting Up CI/CD Pipelines**
This setup includes:
1. **CI Pipeline (Build Stage)** – Exports and packages ADF code.
2. **CD Pipeline (Release Stage)** – Deploys ADF to Dev, QA, and Prod.

---

## **4. CI Pipeline (Build)**
This pipeline triggers on code changes, validates, and exports ADF artifacts.

### **Step 1: Create `azure-pipelines.yml` in Azure DevOps**
```yaml
trigger:
  branches:
    include:
      - main

pool:
  vmImage: 'windows-latest'

variables:
  buildConfiguration: 'Release'
  resourceGroupName: 'my-adf-resource-group'
  dataFactoryName: 'adf-dev'  # Change for each environment

steps:
- task: UsePythonVersion@0
  inputs:
    versionSpec: '3.x'

- script: |
    echo Installing ADF Tools...
    pip install azure-mgmt-datafactory
  displayName: 'Install ADF Management Package'

- task: AzureCLI@2
  inputs:
    azureSubscription: 'Azure Service Connection'
    scriptType: 'bash'
    scriptLocation: 'inlineScript'
    inlineScript: |
      echo Exporting ADF ARM template...
      az datafactory export-template \
        --resource-group $(resourceGroupName) \
        --factory-name $(dataFactoryName) \
        --output-folder adf_publish
  displayName: 'Export ADF Pipeline Template'

- task: PublishBuildArtifacts@1
  inputs:
    pathToPublish: 'adf_publish'
    artifactName: 'drop'


# **5. CD Pipeline (Deployment)**

The **release pipeline** deploys **ADF (Azure Data Factory) pipelines** to **Dev, QA, and Prod** environments.

## **Step 1: Create a Release Pipeline**
1. Navigate to **Azure DevOps > Releases**.
2. Click **New Release Pipeline**.
3. Select **"Empty Job"**.
4. **Add 3 Stages**:
   - **Dev**
   - **QA**
   - **Prod**

---

## **Step 2: Add Deployment Tasks**
For each stage (**Dev, QA, Prod**):

1. Click **"Add a task"** in the selected environment.
2. Select **"Azure Resource Manager Deployment"**.
3. Configure the task with the following details:
   - **Azure Subscription**: Select the service connection.
   - **Resource Group**:
     - **Dev**: `my-adf-dev-rg`
     - **QA**: `my-adf-qa-rg`
     - **Prod**: `my-adf-prod-rg`
   - **Template File**:  
     ```
     $(System.DefaultWorkingDirectory)/_drop/ARMTemplateForFactory.json
     ```
   - **Override Parameters**: Set different **Data Factory Names** for each stage.

---

## **Step 3: Configure Environment-Specific Variables**
1. Click the **"Variables"** tab.
2. Add the following variables for each stage:

| **Environment** | **Resource Group**  | **Data Factory Name** |
|---------------|----------------|----------------|
| **Dev**        | `my-adf-dev-rg` | `adf-dev` |
| **QA**         | `my-adf-qa-rg`  | `adf-qa`  |
| **Prod**       | `my-adf-prod-rg` | `adf-prod` |

---

## **Step 4: Configure Deployment Strategy**
- **Pre-deployment approvals**: Enable manual approval before promoting from **QA → Prod**.
- **Post-deployment Gates**: Add quality checks (e.g., logs, performance metrics).
- **Rollback Policy**: Define rollback actions for failed deployments.

---

## **Step 5: Run the Pipeline**
1. **Commit changes** to the `main` branch.
2. **CI Pipeline exports the ADF templates**.
3. **Release Pipeline automatically deploys** to:
   - ✅ **Dev** (Auto Deployment)
   - ✅ **QA** (Auto Deployment)
   - ✅ **Prod** (Requires Manual Approval)

---

## **Final Workflow Summary**
1. **Developers push code** to Azure DevOps.
2. **CI Pipeline** validates and exports ADF changes.
3. **CD Pipeline** deploys **ADF to Dev → QA → Prod**.

🚀 **Azure DevOps CI/CD is now set up for ADF Deployment!** 🚀


### ARM Templates

# Azure Resource Manager (ARM) Template for Azure Data Factory

## Overview
This ARM template deploys an **Azure Data Factory (ADF)** instance in a specified Azure region.

## **Template JSON**
```json
{
  "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentTemplate.json#",
  "contentVersion": "1.0.0.0",
  "parameters": {
    "factoryName": {
      "type": "string",
      "defaultValue": "myADFInstance",
      "metadata": {
        "description": "The name of the Azure Data Factory instance"
      }
    },
    "location": {
      "type": "string",
      "defaultValue": "[resourceGroup().location]",
      "metadata": {
        "description": "Location for ADF"
      }
    }
  },
  "resources": [
    {
      "type": "Microsoft.DataFactory/factories",
      "apiVersion": "2018-06-01",
      "name": "[parameters('factoryName')]",
      "location": "[parameters('location')]",
      "identity": {
        "type": "SystemAssigned"
      },
      "properties": {},
      "dependsOn": []
    }
  ],
  "outputs": {
    "dataFactoryId": {
      "type": "string",
      "value": "[resourceId('Microsoft.DataFactory/factories', parameters('factoryName'))]"
    }
  }
}
